<a href="https://colab.research.google.com/github/abdelrahmanAh/Sentiment-Analysis/blob/main/sentiment_Analysis_by_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import tensorflow as tf
import numpy as np


In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9685k      0  0:00:08  0:00:08 --:--:-- 15.8M


In [3]:
!tar -xf aclImdb_v1.tar.gz

In [4]:
!rm -r /content/aclImdb/train/unsup

In [5]:
batch_size=32
rw_train_ds=tf.keras.utils.text_dataset_from_directory(
   'aclImdb/train',batch_size=batch_size,validation_split=0.2,subset='training',
    seed=1337
)
rw_val_ds=tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',batch_size=batch_size,validation_split=0.2,subset='validation',
    seed=1337
)
rw_test_ds=tf.keras.utils.text_dataset_from_directory('aclImdb/test',batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [6]:
for text_batch,label_batch in rw_train_ds.take(1):
  for i in range(5):
    print(text_batch.numpy()[i])
    print(label_batch.numpy()[i])


b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

In [7]:
rw_train_ds.class_names

['neg', 'pos']

In [8]:
import string
import re

In [9]:
def custom_standraiztion(input_data):
  lowercase=tf.strings.lower(input_data)
  stripped_html=tf.strings.regex_replace(lowercase,"<br />"," ")
  return tf.strings.regex_replace(stripped_html,f"[{re.escape(string.punctuation)}]","")

In [10]:
max_feature=3000
embedding_dim=128
seq_len=500

In [11]:
vectorize_layer=layers.TextVectorization(standardize=custom_standraiztion,max_tokens=max_feature,
                                         output_mode='int',output_sequence_length=seq_len)

In [12]:
text_ds=rw_train_ds.map(lambda x,y:x)

In [13]:
for i in text_ds.take(1):
  for n in range(2):
    print(i[n])

tf.Tensor(b'I am very disappointed with "K-911." The original "good" quality of "K-9" doesn\'t exist any more. This is more like a sitcom! Some of casts from original movie returned and got some of my memory back. The captain of Dooley now loves to hit him like a scene from old comedy show. That was crazy. What\'s the deal with the change of Police? It seems like they are now LAPD! Not San Diego PD. It is a completely different movie from "', shape=(), dtype=string)
tf.Tensor(b"Giallo fans, seek out this rare film. It is well written, and full of all sorts of the usual low lifes that populate these films. I don't want to give anything away, so I wont even say anything about the plot. The whole movie creates a very bizarre atmosphere, and you don't know what to expect or who to suspect. Recommended! The only place I've seen to get this film in english is from European Trash Cinema, for $15.", shape=(), dtype=string)


In [14]:
vectorize_layer.adapt(text_ds)

In [15]:
def vectorize_text(text,label):
  text=tf.expand_dims(text,-1)
  return vectorize_layer(text),label

In [16]:
train_ds=rw_train_ds.map(vectorize_text)
val_ds=rw_val_ds.map(vectorize_text)
test_ds=rw_test_ds.map(vectorize_text)

In [17]:
train_ds=train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [18]:
test_ds=test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [19]:
val_ds=val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [20]:
inputs=layers.Input(shape=(None,),dtype='int64')
x=layers.Embedding(max_feature,embedding_dim)(inputs)
x=layers.Dropout(0.5)(x)

x=layers.Conv1D(128,7,padding='valid',activation='relu',strides=3)(x)
x=layers.Conv1D(128,7,padding='valid',activation='relu',strides=3)(x)

x=layers.GlobalMaxPooling1D()(x)

x=layers.Dense(256,activation='relu')(x)


x=layers.Dropout(0.5)(x)

prediction=layers.Dense(1,activation='sigmoid',name='predictions')(x)
model=keras.Model(inputs,prediction)
model.compile(loss=keras.losses.BinaryCrossentropy(),optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [21]:
epochs=5
model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/5
625/625 [==============================] - 114s 180ms/step - loss: 0.4711 - accuracy: 0.7436 - val_loss: 0.3226 - val_accuracy: 0.8598
Epoch 2/5
625/625 [==============================] - 108s 173ms/step - loss: 0.2759 - accuracy: 0.8864 - val_loss: 0.3468 - val_accuracy: 0.8580
Epoch 3/5
625/625 [==============================] - 107s 171ms/step - loss: 0.2108 - accuracy: 0.9161 - val_loss: 0.3789 - val_accuracy: 0.8590
Epoch 4/5
625/625 [==============================] - 109s 174ms/step - loss: 0.1593 - accuracy: 0.9382 - val_loss: 0.4384 - val_accuracy: 0.8502
Epoch 5/5
625/625 [==============================] - 108s 174ms/step - loss: 0.1248 - accuracy: 0.9530 - val_loss: 0.4832 - val_accuracy: 0.8466


In [ ]:
epochs=4
model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/4
625/625 [==============================] - 114s 180ms/step - loss: 0.4862 - accuracy: 0.7225 - val_loss: 0.3506 - val_accuracy: 0.8464
Epoch 2/4
625/625 [==============================] - 113s 181ms/step - loss: 0.2172 - accuracy: 0.9135 - val_loss: 0.2966 - val_accuracy: 0.8790
Epoch 3/4
625/625 [==============================] - 118s 189ms/step - loss: 0.1138 - accuracy: 0.9597 - val_loss: 0.4509 - val_accuracy: 0.8500
Epoch 4/4
625/625 [==============================] - 137s 220ms/step - loss: 0.0636 - accuracy: 0.9773 - val_loss: 0.4482 - val_accuracy: 0.8694


In [22]:
model.evaluate(test_ds)


782/782 [==============================] - 35s 44ms/step - loss: 0.4819 - accuracy: 0.8470


[0.4819103181362152, 0.8470399975776672]

In [23]:
def text_analysis(text):


  vec=vectorize_layer(text)
  p=model.predict(tf.expand_dims(vec,0))
  if p>.5 :
    print(f'positive ')
  else :
    print(f'negative ')
  return p

In [24]:
s=text_analysis('I  like the movie')
print(s)

1/1 [==============================] - 0s 289ms/step
positive 
[[0.5793739]]


In [25]:
s=text_analysis('it is very good moviethe movie')
print(s)


1/1 [==============================] - 0s 25ms/step
positive 
[[0.8041344]]


In [26]:
s=text_analysis('it is too much bad movie ')
print(s)


1/1 [==============================] - 0s 33ms/step
negative 
[[0.20571205]]


In [27]:
s=text_analysis('i donot like it')
print(s)


1/1 [==============================] - 0s 26ms/step
positive 
[[0.5255064]]


In [28]:
s=text_analysis('it is impresive ')
print(s)


1/1 [==============================] - 0s 22ms/step
positive 
[[0.55968934]]


In [29]:
s=text_analysis('it is not bad ')
print(s)
#look because we are not use transofrmer or lstm it not recognize the context like
#not bad
#don't like
#like
#so this model is just use word classifcation and not consider the context to much
#this model it just use word like
#(not like it) so it consider it 's positives while the context could be negatives  visversa on negative too

1/1 [==============================] - 0s 26ms/step
positive 
[[0.50638956]]
